## TOOL UPDATED: Dec 22 2022, Henrik Loecke

##Double click this cell to see full description.

##You must restart the kernel after updating Calibration_Variables.py!

<!-- 

To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open Calibration_Variables.py, in the same folder as this notebook, to edit user input there.

All variables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'
generate_confidence_csvs:            Generate csv files for confidence maps, True/False
map_point_spacing:                   Space between dots in confidence maps, e.g. 100 (in meter but number withouth unit)
use_accumulation:                    Use for models with proper tree structure, all models except VSA, True/False
slope_source_unit_meter_per_meter    This is the case for NSSA and FSA, in VSA it is per thousand
model_area_strict_match:             If True, accept 'VSA' but not 'VSA-2019'. If False, accept both.
output_folder:                       Folder path where reports are to be created.
result_folder:                       Folder path of result files.
calibration_sheet:                   Full path, folder included, of calibration parameter sheet.
model:                               Full path, folder included, of model database.
summation_csv:                       Full path, folder included, of summation.csv.
node_csv:                            Full path, folder included, of MH_Zones.csv.
outfall_csv:                         Full path, folder included, of Outfall_Summary.csv.
rainfall_dfs0_file:                  Full path, folder included, of rainfall dfs0 file.
map_folder:                          Folder path where report maps are located.
dfs0_folders:                        Python list of all folders holding measurement dfs0 files.
 -->


In [157]:
#PERMANENT CELL 1

import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
import pickle
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import clear_output
import math
import sqlite3
from Calibration_Variables import *


In [158]:
#PERMANENT CELL 2
#The external script Read_Parameters.py is called to read data from the model.

import subprocess

parameter_script = r"Read_Parameters.py"
bat_file_path = 'Read_Parameters.bat'
bat_file = open(bat_file_path, "w")
bat_file.write('"' + parameter_script + '" "' + os.getcwd() + '" "' + model + '" ' + str(use_accumulation))
bat_file.close()
subprocess.call([bat_file_path]) 




0

In [159]:
#PERMANENT CELL 3
#Import csv files and set up different lists used later in the notebook.

dwf_csv = r"DWF_Specs.csv"
res_types = ['Mixed','ResHD','ResLD']
ici_types = ['Commercial','Industrial','Institutional']
ww_types = res_types + ici_types

all_types = []
for res_type in res_types:
    all_types.append([res_type,'Population','pe','m3/pe/d'])
for ici_type in ici_types:
    all_types.append([ici_type,'Area','ha','m3/ha/d'])
all_types.append(['Baseflow','','',''])
all_types.append(['Total','','',''])
all_types = pd.DataFrame(all_types, columns =['Load_Type','Description','Unit1','Unit2'])
all_types.set_index('Load_Type',inplace=True)

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

dwf_specs = pd.read_csv(os.getcwd() + '\\' + dwf_csv)
dwf_specs.set_index('Zone',inplace=True)

wwf_csv = r"WWF_Specs.csv"

#used to maintain consistent color order for overflows
colors = ['black','purple','green','orange','grey','brown']

gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Shift Y (m)']
if model_area_strict_match == True:
    gauges = gauges[gauges.Model==model_area]
else:
    gauges = gauges[gauges.Model.str.contains(model_area)]
gauges.set_index('Gauge',inplace=True)


periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
if model_area_strict_match == True:
    periods = periods[periods.Model==model_area]
else:
    periods = periods[periods.Model.str.contains(model_area)]



report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
if model_area_strict_match == True:
    report_text = report_text[report_text.Model==model_area]
else:
    report_text = report_text[report_text.Model.str.contains(model_area)]
report_text.fillna('This section was left blank.', inplace=True)

wwf_specs = pd.read_csv(wwf_csv)
wwf_specs.set_index('Location',inplace=True)
wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')

diurnals = pd.read_csv('Diurnals.csv')


if generate_confidence_csvs:
    
    map_periods = pd.read_excel(calibration_sheet,sheet_name="Periods_Map")
    map_periods = map_periods.loc[:, :'Model']
    if model_area_strict_match == True:
        map_periods = map_periods[map_periods.Model==model_area]
    else:
        map_periods = map_periods[map_periods.Model.str.contains(model_area)]

    thresholds = pd.read_excel(calibration_sheet,sheet_name="Thresholds",skiprows=1)
    thresholds.rename(columns={'Unnamed: 0':'Color'},inplace=True)
    for column in thresholds.columns[1:]:
        thresholds.rename(columns={column:column + ' ' + thresholds.loc[0,column]},inplace=True)
    for column in thresholds.columns[1:]:
        thresholds.rename(columns={column:column.replace('.1','')},inplace=True)
        thresholds.rename(columns={column:column.replace('.2','')},inplace=True)
    thresholds.drop(index=0,inplace=True)
    thresholds.drop(thresholds.index[3:],inplace=True) 
    thresholds.sort_values(by='DWF Flow Status',ascending=False,inplace=True)
    thresholds.reset_index(inplace=True)

    map_periods_check = map_periods.loc[:, :'Meter Status']
    map_periods_check_count = map_periods_check[map_periods_check['Meter Status']=='Primary']\
        [['Zone','Meter Status']].groupby(['Zone']).count()
    map_periods_check_count.rename(columns={'Meter Status':'Primary Count'},inplace=True)
    map_periods_check = pd.merge(map_periods_check,map_periods_check_count,on=['Zone'],how='left')
    multiple_primary = list(map_periods_check_count[map_periods_check_count['Primary Count']>1].index)

    missing_primary = []
    for zone in list(gauges.Location.unique()):
        if not zone in (map_periods_check_count.index):
            missing_primary.append(zone)

    if len(multiple_primary) > 0 or len(missing_primary) > 0 :
        error_message = 'Please correct the following errors in ' + os.path.basename(calibration_sheet) + ', sheet Periods_Map.\n\n'
        error_message += 'The following zones have multiple primary meters: '  + ','.join(multiple_primary) + '.\n'
        error_message += 'The following zones have no primary meters: '  + ','.join(missing_primary) + '.\n'
        raise ValueError(error_message)


In [160]:
#PERMANENT CELL 4
#Import manhole spilling from ADDOUT.res1d files.

node_zones = pd.read_csv(node_csv,dtype={'Node': str})
node_zones.set_index(node_zones.Node,inplace=True)
spill_zones = list(node_zones.Zone.unique())

first_round = True
for f in os.listdir(result_folder):
    if '.ADDOUT.res1d' in f:
        print(f)
        res1d = Res1D(result_folder + '\\' + f)
        flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']
        cover_types = ['Normal','Spilling']

        first_round_in_result = True
        res_zones = set()
        for node in res1d.data.Nodes:
            muid = node.Id
            for i, flood_type in enumerate(flood_types):
                ts = res1d.query.GetNodeValues(muid,flood_type)
                col_name = muid + '-' + cover_types[i]
                if ts != None:
                    if max(ts) > 0 or first_round_in_result == True:
                        spill_df = pd.DataFrame(index=res1d.time_index)
                        spill_df['Node'] = muid
                        zone = node_zones.loc[muid,'Zone']
                        spill_df['Zone'] = zone
                        spill_df['Spill'] = ts
                        spill_df['Spill'] = spill_df['Spill'] * 1000
                        
                        res_zones.add(zone)
                        
                        if first_round == True:
                            spill_df_all = spill_df.copy()
                        else:
                            spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                        first_round = False
                        first_round_in_result = False
                        
        #Create empty df for any zone not represented to show 0 on the graph.
        for spill_zone in spill_zones:
            if not spill_zone in res_zones:
                spill_df = pd.DataFrame(index=res1d.time_index)
                spill_df['Node'] = 'X'
                spill_df['Zone'] = spill_zone
                spill_df['Spill'] = 0           
                spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                                                                                      
spill_df_zones = spill_df_all.copy()
spill_df_zones['Date_Time'] = spill_df_zones.index
spill_df_zones.drop(columns=['Node'],inplace=True)
spill_df_zones = spill_df_zones.groupby(['Zone','Date_Time']).sum()
spill_df_zones.reset_index(inplace=True)

                        


DWF_Calib_Aug_3_2017_4_Days_Base.ADDOUT.res1d
DWF_Calib_July_19_2018_5_Days_Base.ADDOUT.res1d
DWF_Calib_Jul_16_2015_4_Days_Base.ADDOUT.res1d
DWF_Calib_Jul_20_2015_3_Days_Base.ADDOUT.res1d
DWF_Calib_May_25_2018_4_Days_Base.ADDOUT.res1d
DWF_Calib_Nov_4_2017_4_Days_Base.ADDOUT.res1d
DWF_Calib_Sep_21_2017_4_Days_Base.ADDOUT.res1d
DWF_November_17_2018_4_Days_Base.ADDOUT.res1d
HotstartBase.ADDOUT.res1d
WWF_Calib_Aug_28_2015_4_Days_Base.ADDOUT.res1d
WWF_Calib_Dec_11_2015_4_Days_Base.ADDOUT.res1d
WWF_Calib_Dec_11_2018_4_Days_Base.ADDOUT.res1d
WWF_Calib_Dec_16_2017_4_Days_Base.ADDOUT.res1d
WWF_Calib_Jan_04_2015_3_Days_Base.ADDOUT.res1d
WWF_Calib_Jan_22_2015_3_Days_Base.ADDOUT.res1d
WWF_Calib_Jul_24_2015_2_Days_Base.ADDOUT.res1d
WWF_Calib_Mar_19_2015_3_Days_Base.ADDOUT.res1d
WWF_Calib_Mar_28_2015_3_Days_Base.ADDOUT.res1d
WWF_Calib_Nov_19_2017_4_Days_Base.ADDOUT.res1d
WWF_Calib_Nov_25_2018_3_Days_Base.ADDOUT.res1d
WWF_Calib_Oct_25_2018_3_Days_Base.ADDOUT.res1d
WWF_Calib_Sep_20_2018_3_Days_Base.AD

In [167]:
#PERMANENT CELL 5
#List all network elements to be imported.

nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Add outfalls
outfalls = pd.read_csv(outfall_csv,dtype={'Weir': str,'Outfall': str})
outfalls['Res_ID'] = ''
for index, row in outfalls.iterrows():
    prefix = ''
    
    if row['Layer'] == 'Summation':
        prefix = ''
    elif row['Layer'] != 'msm_Link':
        prefix = row['Layer'][4:] + ':'
    muid = prefix + row['Weir']
    
    if row['Layer'] != 'Summation':
        outfalls.iloc[index,3] = row['Layer'][4:]
    outfalls.iloc[index,4] = muid
     
    
    if row['Layer'] != 'Summation':
        flow_pipes.append(muid)

catchments = set()
summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})        
for index, row in summation_df.iterrows():
    prefix = ''
    
    if row['Layer'] != 'msm_Link' and row['Layer'] != 'ms_Catchment':
        prefix = row['Layer'][row['Layer'].find('_') + 1:] + ':'

    muid = prefix + row['MUID']
    
    summation_df.iloc[index,0] = muid
    
    if '-Negative' in muid:
        muid = muid[:-9]
    
    if row['Layer'] == 'ms_Catchment':
        catchments.add(muid)
    else:
        flow_pipes.append(muid)
       
#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)


In [162]:
#PERMANENT CELL 6
#Import dfs0 files

#Import rain gauges
rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()
warnings = []

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            
            print(f)
            if 'ps' in f.lower() and f[:2].lower() != 'ps':
                gauge_id = re.split(r'[.]',f)[0]
            else:
                gauge_id = re.split(r'_|[.]',f)[0]
            res = mikeio.read(dfs0_folder + '\\' + f)
            ts = res.to_dataframe()

            gauge_ids.append(gauge_id)
            
            first_level = True
            second_level = True
            first_velocity = True
            for i, column in enumerate(ts.columns):
                if i == 0:
                    ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
                    if 'meter pow 3 per sec' in str(res.items[0]):
                        ts.Flow = ts.Flow * 1000
                    elif 'liter per sec' in str(res.items[0]):
                        ts.Flow = ts.Flow 
                    else:
                        warnings.append('First item in ' + f + ' does not appear to be type Discharge')
                        
                elif str(res.items[i])[-21:] == '<Water Level> (meter)': 
                    if first_level == True:
                        ts.rename(columns={ts.columns[i]:'Level'},inplace=True)
                        first_level = False
                    elif second_level == True:
                        ts.rename(columns={ts.columns[i]:'Level2'},inplace=True)
                        second_level = False
                    

                elif str(res.items[i])[-31:] == '<Flow velocity> (meter per sec)' and first_velocity == True:
                    ts.rename(columns={ts.columns[i]:'Velocity'},inplace=True)
                    first_velocity = False
            
            if not 'Flow' in ts.columns:
                ts['Flow'] = np.nan                     
            if not 'Velocity' in ts.columns:
                ts['Velocity'] = np.nan
            if not 'Level' in ts.columns:
                ts['Level'] = np.nan
            if not 'Level2' in ts.columns:
                ts['Level2'] = np.nan
            
            ts['Gauge'] = gauge_id
            ts = ts[['Gauge','Flow','Level','Level2','Velocity']]
            ts['Seconds'] = ts.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
            ts['Volume'] = ts.Flow * ts.Seconds / 1000
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
    
for warning in warnings:
    print (warning)
    
       

4HI.dfs0
8A1.dfs0
Burrard_PS.dfs0
Cardero_PS.dfs0
Charleson_PS.dfs0
Chilco_PS.dfs0
Clark-Harbour_PS.dfs0
Columbia_PS.dfs0
Glen7th_PS.dfs0
Granville_PS.dfs0
HI1.dfs0
Hudson_PS.dfs0
Jericho_PS.dfs0
Jervis_PS.dfs0
Kent_PS.dfs0
Kitsilano_PS.dfs0
Nelson_PS.dfs0
NEU_PS.dfs0
Raymur_PS.dfs0
Terminal_PS.dfs0
Thornton_PS.dfs0
Willingdon_PS.dfs0
YVR_PS1.dfs0
YVR_PS2.dfs0
YVR_PS24.dfs0
YVR_PS3.dfs0
YVR_PS5.dfs0
YVR_PS6.dfs0
YVR_PS7.dfs0
YVR_PS8.dfs0
YVR_PS9.dfs0
4HI_LEVEL.dfs0
8A1_LEVEL.dfs0
8A25_20150115_20150831_5min_1.dfs0
AG11_20180718_20190130_5min.dfs0
AG5A_20180718_20181125_5min.dfs0
BA4_20150114_20150901_5min.dfs0
BA8_20141222_20150901_5min.dfs0
CA15_20170922_20180215_5min.dfs0
CA24_20170830_20180215_5min.dfs0
CA39_20170914_20180215_5min.dfs0
CA3_.dfs0
CA96_20170914_20180215_5min.dfs0
CA97_20170916_20180215_5min.dfs0
CA98_20170927_20180215_5min.dfs0
CA99_20170914_20180215_5min.dfs0
CC30_20170908_20180215_5min.dfs0
CC6_20170830-20180216.dfs0
CC99_20170908-20180215.dfs0
CC9_20170830_20180215

In [163]:
#PERMANENT CELL 7
#Import network (and runoff if applicable) result files

first_runoff = True
if len(catchments) > 0:
    result_specs = pd.read_csv(result_specs_csv)
    for f in os.listdir(result_folder):
        if 'RR.res1d' in f:
            
            print('Opening ' + f)
            res1d = Res1D(result_folder + '\\' + f)
            
            for catchment in catchments:
                print('Importing ' + catchment)
                catchment_df = pd.DataFrame(index = res1d.time_index)
                catchment_df['ResultFile'] = f
                catchment_df['MUID'] = catchment
                catchment_df['DateTimeRef'] = catchment_df.index
                catchment_df['Discharge'] = res1d.query.GetCatchmentValues(catchment, "TotalRunOff")  
                catchment_df['Discharge'] = catchment_df['Discharge']*1000
               
                catchment_df['Seconds'] = catchment_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                catchment_df['Volume'] = catchment_df.Discharge * catchment_df.Seconds / 1000
                catchment_df.drop(columns=['Seconds'],inplace=True)


                if first_runoff == True:
                    catchment_df_all = catchment_df.copy()
                else:
                    catchment_df_all = pd.concat([catchment_df_all,catchment_df])
                first_runoff = False

result_specs = pd.read_csv(result_specs_csv)
result_specs.rename(columns={'Runoff':'ResultFile'},inplace=True)
catchment_df_all = pd.merge(catchment_df_all,result_specs,how='left',on=['ResultFile'])
catchment_df_all.drop(columns='ResultFile',inplace=True)
catchment_df_all.rename(columns={'Network':'ResultFile'},inplace=True)
catchment_df_all = catchment_df_all[['ResultFile', 'MUID', 'DateTimeRef', 'Discharge', 'Volume']]

results = []
first_level = True
first_flow = True
first_velocity = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f and not 'hotstart' in f.lower():
        res1d = Res1D(result_folder + '\\' + f)
        reaches = res1d.data.Reaches
        nodes = res1d.data.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))

        for i, node in enumerate(nodes):
            
            muid = node.Id
            if muid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                
                level_df = pd.DataFrame(index = res1d.time_index)
                level_df['ResultFile'] = f
                level_df['MUID'] = muid
                level_df['Level'] = res1d.query.GetNodeValues(muid, "WaterLevel")                          

                level_df['DateTimeRef'] = level_df.index

                if first_level == True:
                    level_df_all = level_df.copy()
                else:
                    level_df_all = pd.concat([level_df_all,level_df])
                first_level = False
        
        
        first_round = True
        for i, reach in enumerate(reaches):

            muid = reach.Id[:reach.Id.rfind('-')]

            if muid in us_level_pipes or muid in ds_level_pipes:
            
                print ("Importing pipe " + reach.Id + " level at " + str(dt.datetime.now()))


                if muid in us_level_pipes: 
                    values = res1d.query.GetReachStartValues(muid, "WaterLevel")
                else:
                    values = res1d.query.GetReachEndValues(muid, "WaterLevel")

                level_df = pd.DataFrame(index = res1d.time_index)
                level_df['ResultFile'] = f
                level_df['MUID'] = muid
                level_df['Level'] = values                          

                level_df['DateTimeRef'] = level_df.index

                if first_level == True:
                    level_df_all = level_df.copy()

                else:
                    level_df_all = pd.concat([level_df_all,level_df])
                first_level = False
                        
                        
            
        for i, reach in enumerate(reaches):
            
            muid = reach.Id[:reach.Id.rfind('-')]

            if muid in flow_pipes or muid in list(summation_df.MUID):
                print ("Importing pipe " + reach.Id + " discharge at " + str(dt.datetime.now()))

                values = res1d.query.GetReachEndValues(muid, "Discharge")
                flow_df = pd.DataFrame(index = res1d.time_index)
                flow_df['ResultFile'] = f
                flow_df['MUID'] = muid
                flow_df['DateTimeRef'] = flow_df.index
                flow_df['Discharge'] = values                          
                flow_df['Discharge'] = flow_df['Discharge'] * 1000

                flow_df['Seconds'] = flow_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                flow_df['Volume'] = flow_df.Discharge * flow_df.Seconds / 1000
                flow_df.drop(columns=['Seconds'],inplace=True)

                if first_flow == True:
                    flow_df_all = flow_df.copy()

                else:
                    flow_df_all = pd.concat([flow_df_all,flow_df])
                first_flow = False
                        
        for i, reach in enumerate(reaches):
            
            muid = reach.Id[:(-1 * len(str(i)) - 1)]

            if muid in flow_pipes:
                print ("Importing pipe " + reach.Id + " velocity at " + str(dt.datetime.now()))

                values = res1d.query.GetReachEndValues(muid, "FlowVelocity")
                velocity_df = pd.DataFrame(index = res1d.time_index)
                velocity_df['ResultFile'] = f
                velocity_df['MUID'] = muid
                velocity_df['DateTimeRef'] = velocity_df.index
                velocity_df['Velocity'] = values                          


                if first_velocity == True:
                    velocity_df_all = velocity_df.copy()
                else:
                    velocity_df_all = pd.concat([velocity_df_all,velocity_df])
                first_velocity = False

        first_round = False
        
catchment_df_all_filter  = catchment_df_all.copy()
times = list(flow_df_all.DateTimeRef.unique())
catchment_df_all_filter.set_index(catchment_df_all_filter.DateTimeRef,inplace=True)
catchment_df_all_filter =  catchment_df_all_filter[catchment_df_all_filter['DateTimeRef'].isin(times)]

negatives = list(summation_df.query("MUID.str.endswith('-Negative')").MUID.unique())
original_negatives = []
for negative in negatives:
    original_negatives.append(negative[:-9])
negatives_df = flow_df_all[flow_df_all.MUID.isin(original_negatives)].copy()
negatives_df.MUID = negatives_df.MUID + '-Negative'
negatives_df.Discharge = negatives_df.Discharge * -1
negatives_df.Volume = negatives_df.Volume * -1

flow_df_all = pd.concat([flow_df_all,negatives_df])
        
flow_df_all = pd.concat([flow_df_all,catchment_df_all_filter])
        
#Summation
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum','Volume':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)

flow_df_all = pd.concat([flow_df_all,df_result_sum])

not_founds = []
for muid in summation_df.MUID.unique():
    if not muid in flow_df_all.MUID.unique():
        not_founds.append(muid)
        
if len(not_founds) > 0:
    error_message = 'The following elements in summation_df were not found: \n' 
    error_message += ','.join(not_founds) + '.\n'
    raise ValueError(error_message)


Opening Runoff_2015BaseRR.res1d
Importing 1053_Storm 
Importing 1005_Storm
Importing 1058_Storm
Importing 2205_Storm
Importing 1195
Importing 1071_Storm
Importing 1059_Storm
Importing 1243_Storm
Importing 1208_Storm
Importing 1070_Storm
Importing 1228_Storm
Importing 1212_Storm
Importing 1026_Storm
Importing 1012_Storm
Opening Runoff_2017BaseRR.res1d
Importing 1053_Storm 
Importing 1005_Storm
Importing 1058_Storm
Importing 2205_Storm
Importing 1195
Importing 1071_Storm
Importing 1059_Storm
Importing 1243_Storm
Importing 1208_Storm
Importing 1070_Storm
Importing 1228_Storm
Importing 1212_Storm
Importing 1026_Storm
Importing 1012_Storm
Opening Runoff_2018BaseRR.res1d
Importing 1053_Storm 
Importing 1005_Storm
Importing 1058_Storm
Importing 2205_Storm
Importing 1195
Importing 1071_Storm
Importing 1059_Storm
Importing 1243_Storm
Importing 1208_Storm
Importing 1070_Storm
Importing 1228_Storm
Importing 1212_Storm
Importing 1026_Storm
Importing 1012_Storm
Importing network DWF_Calib_Aug_3_201

Importing pipe 52903A-3748 discharge at 2023-01-04 08:16:10.318193
Importing pipe 53754-3763 discharge at 2023-01-04 08:16:10.324223
Importing pipe 53809-3789 discharge at 2023-01-04 08:16:10.329227
Importing pipe 55237-3927 discharge at 2023-01-04 08:16:10.337256
Importing pipe 55246-3930 discharge at 2023-01-04 08:16:10.342286
Importing pipe 55368-3938 discharge at 2023-01-04 08:16:10.347289
Importing pipe 55462-3942 discharge at 2023-01-04 08:16:10.353316
Importing pipe 55702-3989 discharge at 2023-01-04 08:16:10.359322
Importing pipe 559384-4007 discharge at 2023-01-04 08:16:10.364351
Importing pipe C1-4111 discharge at 2023-01-04 08:16:10.371383
Importing pipe C2-4112 discharge at 2023-01-04 08:16:10.376386
Importing pipe C5_2-4115 discharge at 2023-01-04 08:16:10.381417
Importing pipe FS-27-4235 discharge at 2023-01-04 08:16:10.388422
Importing pipe FS-35-4249 discharge at 2023-01-04 08:16:10.394449
Importing pipe FS-77-4282 discharge at 2023-01-04 08:16:10.400455
Importing pipe 

Importing pipe 515706B-3401 velocity at 2023-01-04 08:16:10.937995
Importing pipe 51583A-3410 velocity at 2023-01-04 08:16:10.942032
Importing pipe 516270-3459 velocity at 2023-01-04 08:16:10.946034
Importing pipe 516486-3489 velocity at 2023-01-04 08:16:10.950037
Importing pipe 51650-3490 velocity at 2023-01-04 08:16:10.953040
Importing pipe 516500-3491 velocity at 2023-01-04 08:16:10.957044
Importing pipe 51692-3544 velocity at 2023-01-04 08:16:10.961047
Importing pipe 51694-3551 velocity at 2023-01-04 08:16:10.964072
Importing pipe 51704-3567 velocity at 2023-01-04 08:16:10.968076
Importing pipe 517370-3592 velocity at 2023-01-04 08:16:10.972110
Importing pipe 51753-3610 velocity at 2023-01-04 08:16:10.976083
Importing pipe 51836-3694 velocity at 2023-01-04 08:16:10.980087
Importing pipe 52903A-3748 velocity at 2023-01-04 08:16:10.984114
Importing pipe 53754-3763 velocity at 2023-01-04 08:16:10.988117
Importing pipe 53809-3789 velocity at 2023-01-04 08:16:10.991120
Importing pipe 55

Importing node WW-7 at 2023-01-04 08:16:14.889661
Importing node WW-8 at 2023-01-04 08:16:14.895690
Importing node WW-9 at 2023-01-04 08:16:14.901726
Importing pipe 1450302-89 discharge at 2023-01-04 08:16:14.923802
Importing pipe 1523387-185 discharge at 2023-01-04 08:16:14.931837
Importing pipe 1545519-209 discharge at 2023-01-04 08:16:14.938842
Importing pipe 1641277-271 discharge at 2023-01-04 08:16:14.945875
Importing pipe 40269-338 discharge at 2023-01-04 08:16:14.953902
Importing pipe 40271-339 discharge at 2023-01-04 08:16:14.961411
Importing pipe 40543-366 discharge at 2023-01-04 08:16:14.968450
Importing pipe 40547-370 discharge at 2023-01-04 08:16:14.975489
Importing pipe 40553-376 discharge at 2023-01-04 08:16:14.982521
Importing pipe 40600-418 discharge at 2023-01-04 08:16:14.990528
Importing pipe 40637-440 discharge at 2023-01-04 08:16:14.997562
Importing pipe 40642-445 discharge at 2023-01-04 08:16:15.004611
Importing pipe 40649-450 discharge at 2023-01-04 08:16:15.01164

Importing pipe Weir:Weir_Maple&York-4706 discharge at 2023-01-04 08:16:15.956902
Importing pipe Orifice:Dunbar_49th_Flap_Gate-4858 discharge at 2023-01-04 08:16:15.965940
Importing pipe Orifice:Dunbar_Intake_Upper-4860 discharge at 2023-01-04 08:16:15.973972
Importing pipe Orifice:Highbury_to_4th_Siphon_1-4867 discharge at 2023-01-04 08:16:15.982003
Importing pipe Orifice:Highbury_to_4th_Siphon_2-4868 discharge at 2023-01-04 08:16:15.990009
Importing pipe Orifice:Hudson_Flap_Gate-4870 discharge at 2023-01-04 08:16:15.998046
Importing pipe Orifice:Orifice_Kitsilano_PS-4923 discharge at 2023-01-04 08:16:16.006080
Importing pipe Orifice:Orifice_NAI_ANGUS_SIPHON-4926 discharge at 2023-01-04 08:16:16.014110
Importing pipe Orifice:Orifice_NAI_SIPHON_OVERFLOW-4932 discharge at 2023-01-04 08:16:16.022147
Importing pipe Orifice:Willingdon_PS_Overflow-4958 discharge at 2023-01-04 08:16:16.030152
Importing pipe 1450302-89 velocity at 2023-01-04 08:16:16.038159
Importing pipe 1523387-185 velocity 

...

Importing pipe Weir:ADO to EBO Diversion Weir-4631 discharge at 2023-01-04 08:18:56.744235
Importing pipe Weir:Angus Drive Overflow Weir-4633 discharge at 2023-01-04 08:18:56.781269
Importing pipe Weir:Arbutus_To_McNicol_Weir-4635 discharge at 2023-01-04 08:18:56.818391
Importing pipe Weir:Balaclava & Point Gray_85307-4636 discharge at 2023-01-04 08:18:56.856425
Importing pipe Weir:Bladder_Gate_East-4639 discharge at 2023-01-04 08:18:56.893514
Importing pipe Weir:Bladder_Gate_West-4640 discharge at 2023-01-04 08:18:56.930568
Importing pipe Weir:Borden-Kent Av Overflow Weir-4644 discharge at 2023-01-04 08:18:56.967629
Importing pipe Weir:BRIS019 BRIG-CASSIAR OUTF-4645 discharge at 2023-01-04 08:18:57.005663
Importing pipe Weir:Brockton_Weir-4646 discharge at 2023-01-04 08:18:57.042757
Importing pipe Weir:CAMBIE & EBI-4648 discharge at 2023-01-04 08:18:57.079820
Importing pipe Weir:Cassiar_Relief_Weir-4649 discharge at 2023-01-04 08:18:57.116911
Importing pipe Weir:CODR088 COMMERCIALtoCL

Importing pipe 52903A-3748 velocity at 2023-01-04 08:19:00.552019
Importing pipe 53754-3763 velocity at 2023-01-04 08:19:00.583048
Importing pipe 53809-3789 velocity at 2023-01-04 08:19:00.615076
Importing pipe 55237-3927 velocity at 2023-01-04 08:19:00.646183
Importing pipe 55246-3930 velocity at 2023-01-04 08:19:00.679305
Importing pipe 55368-3938 velocity at 2023-01-04 08:19:00.711411
Importing pipe 55462-3942 velocity at 2023-01-04 08:19:00.742434
Importing pipe 55702-3989 velocity at 2023-01-04 08:19:00.774463
Importing pipe 559384-4007 velocity at 2023-01-04 08:19:00.806519
Importing pipe C1-4111 velocity at 2023-01-04 08:19:00.837547
Importing pipe C2-4112 velocity at 2023-01-04 08:19:00.869576
Importing pipe C5_2-4115 velocity at 2023-01-04 08:19:00.900657
Importing pipe FS-27-4235 velocity at 2023-01-04 08:19:00.932737
Importing pipe FS-35-4249 velocity at 2023-01-04 08:19:00.963794
Importing pipe FS-77-4282 velocity at 2023-01-04 08:19:00.994843
Importing pipe FS-890-4285 vel

Importing node 9259 at 2023-01-04 08:19:05.658060
Importing node 9277 at 2023-01-04 08:19:05.686193
Importing node 9284 at 2023-01-04 08:19:05.706211
Importing node 9301 at 2023-01-04 08:19:05.724288
Importing node 9306 at 2023-01-04 08:19:05.742341
Importing node 9327 at 2023-01-04 08:19:05.759357
Importing node 9356 at 2023-01-04 08:19:05.779375
Importing node 9396 at 2023-01-04 08:19:05.796415
Importing node 9420 at 2023-01-04 08:19:05.813458
Importing node 9459 at 2023-01-04 08:19:05.830493
Importing node 9464 at 2023-01-04 08:19:05.849534
Importing node WW-1 at 2023-01-04 08:19:05.866570
Importing node WW-2 at 2023-01-04 08:19:05.885588
Importing node WW-24 at 2023-01-04 08:19:05.904604
Importing node WW-3 at 2023-01-04 08:19:05.924622
Importing node WW-5 at 2023-01-04 08:19:05.942665
Importing node WW-6 at 2023-01-04 08:19:05.960681
Importing node WW-7 at 2023-01-04 08:19:05.977697
Importing node WW-8 at 2023-01-04 08:19:05.997740
Importing node WW-9 at 2023-01-04 08:19:06.014779

Importing pipe Weir:Bladder_Gate_East-4639 discharge at 2023-01-04 08:19:10.158574
Importing pipe Weir:Bladder_Gate_West-4640 discharge at 2023-01-04 08:19:10.197635
Importing pipe Weir:Borden-Kent Av Overflow Weir-4644 discharge at 2023-01-04 08:19:10.235719
Importing pipe Weir:BRIS019 BRIG-CASSIAR OUTF-4645 discharge at 2023-01-04 08:19:10.272805
Importing pipe Weir:Brockton_Weir-4646 discharge at 2023-01-04 08:19:10.308931
Importing pipe Weir:CAMBIE & EBI-4648 discharge at 2023-01-04 08:19:10.344988
Importing pipe Weir:Cassiar_Relief_Weir-4649 discharge at 2023-01-04 08:19:10.381021
Importing pipe Weir:CODR088 COMMERCIALtoCLARK-4653 discharge at 2023-01-04 08:19:10.418054
Importing pipe Weir:CroweSt_OF_Weir_1-4655 discharge at 2023-01-04 08:19:10.455088
Importing pipe Weir:CroweSt_OF_Weir_2-4656 discharge at 2023-01-04 08:19:10.494124
Importing pipe Weir:DIVERSION AT YUKON & 8TH-4657 discharge at 2023-01-04 08:19:10.533159
Importing pipe Weir:Gilford_Weir-4661 discharge at 2023-01-0

Importing pipe 55237-3927 velocity at 2023-01-04 08:19:14.080275
Importing pipe 55246-3930 velocity at 2023-01-04 08:19:14.111303
Importing pipe 55368-3938 velocity at 2023-01-04 08:19:14.143332
Importing pipe 55462-3942 velocity at 2023-01-04 08:19:14.173359
Importing pipe 55702-3989 velocity at 2023-01-04 08:19:14.206389
Importing pipe 559384-4007 velocity at 2023-01-04 08:19:14.235416
Importing pipe C1-4111 velocity at 2023-01-04 08:19:14.267445
Importing pipe C2-4112 velocity at 2023-01-04 08:19:14.297542
Importing pipe C5_2-4115 velocity at 2023-01-04 08:19:14.330573
Importing pipe FS-27-4235 velocity at 2023-01-04 08:19:14.362640
Importing pipe FS-35-4249 velocity at 2023-01-04 08:19:14.393763
Importing pipe FS-77-4282 velocity at 2023-01-04 08:19:14.423833
Importing pipe FS-890-4285 velocity at 2023-01-04 08:19:14.456864
Importing pipe GS-660-4356 velocity at 2023-01-04 08:19:14.489010
Importing pipe KWL-1-4380 velocity at 2023-01-04 08:19:14.519639
Importing pipe Link_1178-4381

In [168]:
#PERMANENT CELL 8
#Create HTML reports

error_list = []

for gauge in gauges.index:
    if gauge in list(periods.Meter):

        location_type = gauges.loc[gauge,'Location Type']
        zone = str(gauges.loc[gauge,'Location'])
        pipe = str(gauges.loc[gauge,'Pipe'])
        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
        rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
        MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
        NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
        MHName = str(gauges.loc[gauge,'Node1 AssetName']) 
        number_catchments = wwf_specs.loc[zone,'Number sanitary'] \
            + wwf_specs.loc[zone,'Number combined'] \
            + wwf_specs.loc[zone,'Number stormwater']
        
        effluent_types = []
        if wwf_specs.loc[zone,'Number sanitary'] > 0:
            effluent_types.append('Sanitary')
        if wwf_specs.loc[zone,'Number combined'] > 0:
            effluent_types.append('Combined')
        if wwf_specs.loc[zone,'Number stormwater'] > 0:
            effluent_types.append('Storm')
        
        for i, effluent_type in enumerate(effluent_types):
            if i == 0:
                effluent_type_string = effluent_type
            elif i == len(effluent_types)-1:
                effluent_type_string += ' and ' + effluent_type
            else:
                effluent_type_string += ', ' + effluent_type          
            
                    
        local_population = 0
        for ww_type in ww_types:
            if ww_type in res_types:                
                local_population += dwf_specs.loc[zone,ww_type + '_Population']
                
        
        total_area = round(wwf_specs.loc[zone,'Drainage area (ha)'],1)

        PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
        try:
            PipeDimension = str(round(gauges.loc[gauge, 'Pipe Dimension'],2))
        except:
            PipeDimension = gauges.loc[gauge, 'Pipe Dimension']


        with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
            
            f.write('<style>')
#             f.write('@page {margin-left: 5cm;}')
            f.write('p    {width:1000px;}')
            f.write('p    { font-size: 20px;}')
#---------------------------------------------------------------------------------------------------------------------        
 
    
            f.write('* {\n')
            f.write('  box-sizing: border-box;\n')
            f.write('}\n')

            f.write('.row {\n')
            f.write('  margin-left:0px;\n')
            f.write('  margin-right:-5px;\n')

            f.write('}\n')

            f.write('.column {\n')
            f.write('  float: left;\n')
            f.write('  width: 17%;\n')
            f.write('  padding: 5px;\n')
            f.write('}\n')

            f.write('/* Clearfix (clear floats) */\n')
            f.write('.row::after {\n')
            f.write('  content: "";\n')
            f.write('  clear: both;\n')
            f.write('  display: table;\n')
            f.write('}\n')

            f.write('table.first, table.first th, table.first td {\n')
            f.write('  border: 2px solid;\n')
            f.write('  padding: 4px;\n')

            f.write('  border-collapse: collapse;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 800px;\n')
            f.write('}\n') 

            f.write('table.second {\n')
            f.write('  border: 2px solid;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 310px;\n')
            f.write('  padding: 1px;\n')
            f.write('  margin: 1px;\n')

            f.write('})\n')    
    
#---------------------------------------------------------------------------------------------------------------------        
             
#             f.write('table.grid, th, td {')
#             f.write('  border: 1px solid;')
#             f.write('  padding: 4px;')
#             f.write('  border-collapse: collapse;')
            
            f.write('})')            
            f.write('</style>')
            f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                    + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')

            f.write('<h2>1. Description of the Calibration Area</h2>')
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>Number of Catchments</td>\n')
            f.write('<td>'+ str(number_catchments) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Zone Area (Ha)</td>\n')
            f.write('<td>'+ str(total_area) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Local Residential Population</td>\n')
            f.write('<td>'+ str(int(local_population)) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Effluent Type</td>\n')
            f.write('<td>'+ effluent_type_string +'</td>\n')
            f.write('</tr>\n')            
            for ww_type in ww_types:
                if ww_type not in res_types:
                    f.write('<tr>\n')
                    f.write('<td>' + ww_type + ' Area (Ha)</td>\n')
#                     f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_' + description],1)) +'</td>\n')
                    f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_Area'],1)) +'</td>\n')
                    f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('</table>')
            
            f.write('<h2>2. Description of the Sensor and Network</h2>')            
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Monitoring Type' +'</td>\n')
            f.write('<td>'+ MonitoringType +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Node ID' +'</td>\n')
            f.write('<td>'+ node +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe ID' +'</td>\n')
            f.write('<td>'+ pipe +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Shape' +'</td>\n')
            f.write('<td>'+ PipeShape +'</td>\n')
            f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
            f.write('<td>'+ PipeDimension +'</td>\n')
            f.write('</tr>\n')
            f.write('</table>')
    
            
            f.write('<h2>3. Model Calibration Setup: </h2>')
                    
            f.write('<table class="first">\n')
            f.write('  <tr>\n')
            f.write('    <th>Simulation</th>\n')
            f.write('    <th>Start Date</th>\n')
            f.write('    <th>End Date</th>\n')
            f.write('    <th>Duration</th>\n')
            f.write('  </tr>\n')

            for period_spec in period_specs:
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                if pd.isnull(start):
                    break
                duration = str((end - start).days) + ' days'
                
                f.write('  <tr>\n')
                f.write('    <td>' + description + '</th>\n')
                f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + duration + '</th>\n')
                f.write('  </tr>\n')                   
            f.write('</table>\n')
                                      
            f.write('<h2>4. Zone Map</h2>\n')
            f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">\n')
            
            f.write('<h2>6. Diurnal Patterns</h2>\n')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Diurnal Text'])[0]) + '</p>\n')
            fig_diurnal = go.Figure()
            diurnals_filter1 = diurnals[diurnals.Zone==zone]
            profile_ids = list(diurnals_filter1.Profile.unique())
            for profile_id in profile_ids:
                diurnals_filter2 = diurnals_filter1[diurnals_filter1.Profile==profile_id]
                fig_diurnal.add_trace(go.Scatter(x=diurnals_filter2.Sqn, 
                                             y = diurnals_filter2.Multiplier, 
                                             mode='lines',name=profile_id + ' (' + diurnals_filter2.iloc[0,2] + ')'))             
            fig_diurnal.update_layout(
                autosize=False,
                width = 1362,
                height=250,
                margin=dict(
                    l=50,
                    r=50,
                    b=25,
                    t=25,
                    pad=4
                    ),
#                 yaxis_title="Discharge (L/s)" 
                )
            
            f.write(fig_diurnal.to_html(full_html=False, include_plotlyjs='cdn'))
            
            f.write('<h2>6. DWF Calibration Summary</h2>\n')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>\n')
            f.write('<h2>7. WWF Calibration and Validation Summary</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>\n')
            f.write('<h2>8. Calibration Issues</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>\n')
            f.write('<h2>9. Recommendations</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>\n')
            f.write('<h2>10. DWF Calibration Plots and Statistics</h2>\n')

            wwf_plot_header_added = False
            for period_spec in period_specs:
                
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                
                if pd.isnull(start):
                    break

                if period_spec[:3] == "WWF" and wwf_plot_header_added == False:
                    f.write('<h2>11. WWF Calibration Plots</h2>\n')
                    wwf_plot_header_added = True                                                

                chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

                measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                
                if location_type == 'PS':
                    measured_gauge['Discharge_Hourly'] = measured_gauge['Flow'].rolling('1h').mean()
                
                measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                
                result_velocity_gauge = velocity_df_all.loc[(velocity_df_all.MUID==pipe) & (velocity_df_all.index >= start) & (velocity_df_all.index <= end)].copy()
                
                spill_df_zone = spill_df_zones.loc[(spill_df_zones.Zone==zone) & (spill_df_zones.Date_Time >= start) & (spill_df_zones.Date_Time <= end)].copy()                
                spill_df_all_zone = spill_df_all.loc[(spill_df_all.Zone==zone) & (spill_df_all.index >= start) & (spill_df_all.index <= end)].copy()
                spill_df_all_zone.drop(columns=['Zone'],inplace=True)
                spill_top_ten = spill_df_all_zone.groupby(['Node']).max()
                spill_top_ten.reset_index(inplace=True)
                spill_top_ten.sort_values(by='Spill',ascending=False,inplace=True)
                spill_top_ten.reset_index(drop=True,inplace=True)
                spill_top_ten = spill_top_ten[spill_top_ten['Spill']>0]
                spill_top_ten = spill_top_ten.head(10)                                
                
                if location_type == 'PS':
                    result_flow_gauge['Discharge_Hourly'] = result_flow_gauge['Discharge'].rolling('1h').mean()
                
                result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

                has_outfalls = False
                if zone in list(outfalls.Zone):
                    has_outfalls = True
                    outfalls_filter = outfalls[outfalls.Zone==zone].copy()
                    outfalls_filter.reset_index(inplace=True)

                    for index, row in outfalls_filter.iterrows():
                        overflow_pipe = row['Res_ID']
                        result_outfalls = flow_df_all.loc[(flow_df_all.MUID==overflow_pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                        result_outfalls['Outfall'] = row['Outfall']
                        if index == 0:
                            result_outfalls_all = result_outfalls.copy()
                        else:
                            result_outfalls_all = pd.concat([result_outfalls_all,result_outfalls])
                
                
            #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]
                #---------------------------------------------------------------------------------------------
                compare_stats = []
                peak_level_model = round(result_level_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Model',peak_level_model,'m'])
                peak_level_measured = round(measured_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Measured',peak_level_measured,'m'])
                compare_stats.append(['Peak Level Difference',round(peak_level_model - peak_level_measured,2),'m'])
#                 volume_model = round(result_flow_gauge.Volume.sum(),0).item()
                volume_model = round(result_flow_gauge.Volume.sum(),0)
                compare_stats.append(['Volume Model',volume_model,'m3'])
                volume_measured = round(measured_gauge.Volume.sum(),0).item() #convert to native float to avoid inf
                compare_stats.append(['Volume Measured',volume_measured,'m3'])
                try:
                    compare_stats.append(['Volume Difference',round((volume_model - volume_measured) / volume_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Volume Difference','','%'])
                peak_flow_model = round(result_flow_gauge.Discharge.max(),1)
                compare_stats.append(['Peak Flow Model',peak_flow_model,'L/s'])
                peak_flow_measured = round(measured_gauge.Flow.max(),1)
                compare_stats.append(['Peak Flow Measured',peak_flow_measured,'L/s'])
                try:
                    compare_stats.append(['Peak Flow Difference',round((peak_flow_model - peak_flow_measured) / peak_flow_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Peak Flow Difference','','%'])
                
                
                f.write('<h3>' + chart_header + '</h3>\n')  
                #----------------------------------------------------------------------------------------------------   
                if period_spec[:3] == "DWF":
                    
                    f.write('<div class="row">')
                    
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Description</th>\n')
                    f.write('    <th>Value</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')                   
                    
                    #sim and measured stats
                    for compare_stat in compare_stats:
                        f.write('  <tr>\n')
                        f.write('    <td>' + compare_stat[0] + '</th>\n')
                        f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                        f.write('    <td>' + compare_stat[2] + '</th>\n')
                        f.write('  </tr>\n')
                        
                    f.write('</table>\n')
                    f.write('</div>')                    
                    
                    #Loading rate
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Loading Rate</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')
                                       
                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 3
                        else: 
                            decimals = 1
                        rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                        unit = all_types.loc[ww_type,'Unit2']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(rate) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')

                    #Population and ICI areas
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    if use_accumulation == True:
                        f.write('    <th>Total</th>\n')
                    else:
                        f.write('    <th></th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 0
                        else: 
                            decimals = 1
                        description = unit = all_types.loc[ww_type,'Description']
                        local_val = round(dwf_specs.loc[zone,ww_type + '_' + description],decimals)
                        if use_accumulation == True:
                            total_val = round(dwf_specs.loc[zone,ww_type + '_' + description + '_Upstream'],decimals)
                        unit = all_types.loc[ww_type,'Unit1']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        if use_accumulation == True:
                            f.write('    <td>' + str(total_val) + '</th>\n')
                        else:
                            f.write('    <td></th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')
                    #Waterload
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    if use_accumulation == True:
                        f.write('    <th>Total</th>\n')
                    else:
                        f.write('    <th></th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for load_type in all_types.index.values.tolist():

                        local_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad'],0)
                        if use_accumulation == True:
                            total_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad_Upstream'],0)
                        unit = 'm3/d'

                        f.write('  <tr>\n')
                        f.write('    <td>' + load_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        if use_accumulation == True:
                            f.write('    <td>' + str(total_val) + '</th>\n')
                        else:
                            f.write('    <td></th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                        
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')

                    f.write('</table>\n')
                    f.write('</div>')
#                     f.write('</div>')
                    
                else:

                    total_rows = 10
                    f.write('<div class="row">')
                    for i in range(0,4): #Cycle through the three table types.   
                        f.write('<div class="column">')    
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Description</th>\n')
                        f.write('    <th>Value</th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        current_row = 0
                        if i == 0:
                            for compare_stat in compare_stats:
                                f.write('  <tr>\n')
                                f.write('    <td>' + compare_stat[0] + '</th>\n')
                                f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                                f.write('    <td>' + compare_stat[2] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1                
                        else:
                            for index, row in wwf_stats_specs[wwf_stats_specs.Table==i].iterrows():

                                decimals = row['Decimals']

#                                 rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
#                                 unit = all_types.loc[ww_type,'Unit2']


                                f.write('  <tr>\n')
                                f.write('    <td>' + row['Description'] + '</th>\n')
                                if row['DF'] == 'wwf_specs':
                                    if slope_source_unit_meter_per_meter == False and row['Description'] == 'Average slope':
                                        f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']]/1000,decimals)) + '</th>\n')
                                    elif decimals > 0:
                                        f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']],decimals)) + '</th>\n')
                                    else:
                                        f.write('    <td>' + str(int(wwf_specs.loc[zone,row['DF_Col']])) + '</th>\n')
                                else: 
                                    f.write('    <td>' + str(round(measured_rain[rain_gauge].sum(),decimals)) + '</th>\n')
                                f.write('    <td>' + row['Unit'] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1

                        remaining_rows = total_rows - current_row

                        for remaining_row in range(remaining_rows):
                            f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('</table>\n')
                        f.write('</div>')
                        
                #Spill top 10
                if period_spec[:3] == "DWF":
                    total_rows = 9
                else:
                    total_rows = 10
                    
                f.write('<div class="row">')
                  
                f.write('<div class="column">')    
                f.write('<table class="second">\n')
                f.write('  <tr>\n')
                f.write('    <th>Node Spill Top 10</th>\n')
                f.write('    <th>Peak Spill (L/s)</th>\n')
                f.write('  </tr>\n')

                current_row = 0

                for index, row in spill_top_ten.iterrows():
                    f.write('  <tr>\n')
                    f.write('    <td>' + row['Node'] + '</th>\n')
                    f.write('    <td>' + str(int(row['Spill'])) + '</th>\n')
                    f.write('  </tr>\n')
                    current_row += 1                


                remaining_rows = total_rows - current_row

                for remaining_row in range(remaining_rows):
                    f.write('<tr><td><br/></td><td><br/></td></tr>\n')
                f.write('</table>\n')
                f.write('</div>')
        

                f.write('</div>')
                    #----------------------------------------------------------------------------------------------


                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Discharge at Gauge ' + gauge + ', Reach ' + pipe + ', Rain Gauge ' + rain_gauge,  
                                                    'Water Level at Gauge ' + gauge + ', Node ' + node),
                                                    specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                                                   horizontal_spacing = 0.1)

                fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)

                if location_type == 'PS':
                    fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge_Hourly, mode='lines',name='Model Flow Hourly',line_color='yellow'),1,1)   
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Discharge_Hourly, mode='lines',name='Measured Flow Hourly',line_color='black'),1,1)
                
                fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level',line_color='green'),1,2)
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level',line_color='purple'),1,2)
                           
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ),
                    title ={
                        'text' : chart_header,
                        'x':0.45,
                        'xanchor': 'center'
                    })
                
                fig['layout']['yaxis']['title']='Dicharge (L/s)'
                fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
                fig['layout']['yaxis2']['range']=[10,0]
                fig['layout']['yaxis3']['title']='Water Level (m)'
                
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')
                
                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Velocity at Gauge ' + gauge + ', Reach ' + pipe,
                                                    'Total Manhole Spills, CSO and SSO'),
                                                    specs=[[{"secondary_y": False}, {"secondary_y": False}]],
                                                   horizontal_spacing = 0.1)
                
                fig.add_trace(go.Scatter(x=result_velocity_gauge.DateTimeRef, y=result_velocity_gauge.Velocity, mode='lines',name='Model Velocity'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Velocity, mode='lines',name='Measured Velocity'),1,1)
                
                spill_df_zone
                fig.add_trace(go.Scatter(x=spill_df_zone.Date_Time, y=spill_df_zone.Spill, mode='lines',name='Total Manhole Spilling'),1,2)
                if has_outfalls == True:
                
                    show_first = []
                    show_last = []
                    for outfall in list(outfalls_filter.Outfall):
                        if outfall in measured.Gauge.unique():
                            show_first.append(outfall)
                        else:
                            show_last.append(outfall)
                    outfalls_to_plot = show_first + show_last
                    for i, outfall in enumerate(outfalls_to_plot):
                        result_outfalls_all_filter = result_outfalls_all[result_outfalls_all.Outfall==outfall]
                        if i < len(colors):
                            fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                                     y = result_outfalls_all_filter.Discharge, 
                                                     mode='lines',name=outfall,line={'color': colors[i]}),1,2)
                        else:
                            fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                                     y = result_outfalls_all_filter.Discharge, 
                                                     mode='lines',name=outfall),1,2)
                         
                        if outfall in measured.Gauge.unique():
                            outfall_measured = measured.loc[(measured.Gauge==outfall) & (measured.index >= start) & (measured.index <= end)].copy()    
                            if i < len(colors):
                                fig.add_trace(go.Scatter(x=outfall_measured.index, 
                                                         y=outfall_measured.Flow, 
                                                         mode='lines',name=outfall + ' - Measured',
                                                         line={'color': colors[i],'dash': 'dash'}),1,2)                                                         
                            else:
                                fig.add_trace(go.Scatter(x=outfall_measured.index, 
                                                         y=outfall_measured.Flow, 
                                                         mode='lines',name=outfall + ' - Measured',
                                                         line={'dash': 'dash'}),1,2)
                                                                                         
                
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ))
#                     title ={
#                         'text' : chart_header,
#                         'x':0.45,
#                         'xanchor': 'center'
#                     })
                
                fig['layout']['yaxis']['title']='Velocity (m/s)'
                fig['layout']['yaxis2']['title']='Dicharge (L/s)'
                
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')
                          

        f.close()
        print("Writing html for " + gauge)
#     except Exception as e:
#         error_list.append([gauge_id,e])

    

Writing html for 8A25
Writing html for AG11
Writing html for AG5A
Writing html for BA4
Writing html for BA8
Writing html for Burrard_PS
Writing html for CA15
Writing html for CA24
Writing html for CA39
Writing html for CA96
Writing html for CA97
Writing html for CA98
Writing html for CA99
Writing html for Cardero_PS
Writing html for CC30
Writing html for CC6
Writing html for CC9
Writing html for CC99
Writing html for Charleson_PS
Writing html for Chilco_PS
Writing html for Clark-Harbour_PS
Writing html for CO1
Writing html for CO11
Writing html for Columbia_PS
Writing html for CY10
Writing html for EB3
Writing html for Glen7th_PS
Writing html for HA20
Writing html for HA22
Writing html for HD3
Writing html for HE4
Writing html for HE40
Writing html for HR3
Writing html for HR4
Writing html for Hudson_PS
Writing html for HW9
Writing html for Jericho_PS
Writing html for Jervis_PS
Writing html for KA18
Writing html for Kent_PS
Writing html for Kitsilano_PS
Writing html for MD4
Writing htm

KeyboardInterrupt: 

In [165]:
round(result_flow_gauge.Volume.sum(),0)

397.0

In [166]:
#PERMANENT CELL 9
#Create csv files for confidence maps.

if generate_confidence_csvs == True:

    color_dict = {}
    color_dict[1] = 'Green'
    color_dict[1.5] = 'Light Green'
    color_dict[2] = 'Yellow'
    color_dict[2.5] = 'Light Red'
    color_dict[3] = 'Red'


    event_categories = ['DWF Calibration','WWF Calibration','WWF Validation']

    result_types = ['HGL','Flow','Volume']

    result_points = []
    zone_review = []

    for index, row in map_periods.iterrows():
        gauge = row['Meter']
        zone = row['Zone']
        meter_status = row['Meter Status']
        x = gauges.loc[gauge,'X coordinate']
        y = gauges.loc[gauge,'Y coordinate']
        x_shift = gauges.loc[gauge,'Shift X (m)']
        y_shift = gauges.loc[gauge,'Shift Y (m)']
        catchment_type = gauges.loc[gauge,'Catchment Type']

        print('Processing ' + gauge + ' for confidence maps.')

        for i, event_category in enumerate(event_categories):
            event_category_short = event_category[:7]
            start = row[event_category_short + ' Start']
            end = row[event_category_short + ' End']
            y_minor_shift = -i * map_point_spacing
            y_final = y + y_shift + y_minor_shift

            color_values = []
            for j, result_type in enumerate(result_types):
                x_minor_shift = j * map_point_spacing
                x_final = x + x_shift + x_minor_shift
                bad_data = False
                if result_type == 'HGL':
                    result_type_csv = 'Max HGL'
                    qa = row[event_category_short + ' WL QA']
                    if qa.lower() == 'bad data':
                        bad_data = True
                    else:
                        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
                        max_model = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)].Level.max()
                        max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Level.max()
                        if not math.isnan(max_measured) and max_measured != 0:
                            diff = abs(max_model - max_measured) * 100 #cm
                        else:
                            diff = 'NA'

                elif result_type == 'Flow':
                    result_type_csv = 'Max Flow'
                    qa = row[event_category_short + ' Flow QA']
                    if qa.lower() == 'bad data':
                        bad_data = True
                    else:
                        pipe = str(gauges.loc[gauge,'Pipe'])
                        filter_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                        filter_model['Hourly'] = filter_model.Discharge.rolling('1h').mean()
                        filter_model = filter_model.iloc[12: , :] #Remove first hour to not get skewed hourly average there                        
                        max_model = filter_model.Hourly.max()

                        filter_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                        filter_measured['Hourly'] = filter_measured.Flow.rolling('1h').mean()
                        filter_measured = filter_measured.iloc[12: , :] #Remove first hour to not get skewed hourly average there 
                        max_measured = filter_measured.Hourly.max()
                        if not math.isnan(max_measured) and max_measured != 0:
                            diff = abs(max_model - max_measured) / max_measured
                        else:
                            diff = 'NA'

                else:
                    result_type_csv = 'Acc Volume'
                    qa = row[event_category_short + ' Flow QA']
                    if qa.lower() == 'bad data':
                        bad_data = True
                    else:
                        pipe = str(gauges.loc[gauge,'Pipe'])
                        max_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].Volume.sum()
                        max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Volume.sum()
                        if not math.isnan(max_measured) and max_measured != 0:
                            diff = abs(max_model - max_measured) / max_measured
                        else:
                            diff = 'NA'

                if bad_data == True:
                    color = 'Black'
                    color_value = 0
                elif diff == 'NA':
                    color = 'Grey'
                    color_value = 0
                else:
                    if event_category[:3] == 'WWF':
                        threshold_column = event_category[:3] + ' ' + catchment_type + ' ' + result_type + ' Status'
                    else:
                        threshold_column = event_category[:3] + ' ' + result_type + ' Status'
                    for k in range(3):
                        threshold = thresholds.loc[k,threshold_column]
                        if diff >= threshold:
                            color = thresholds.loc[k,'Color']
                            color_value = thresholds.loc[k,'index']
                            break

                color_values.append(color_value)

                label = ''
                if i == 0 and j == 0:
                    label = gauge

                result_points.append([gauge,x_final,y_final,label,event_category,result_type_csv,color])

            if meter_status == 'Primary':
                weighting_components = []
                weighted_color_value = 0
                value_count = 0
                qa = 'OK'
                for k, color_value in enumerate(color_values):
                    if color_value > 0:
                        if k == 0:
                            weighting_components.append(color_value) #Level is weighted as 50%
                            value_count += 1
                        else:
                            weighting_components.append(color_value/2) #Flow is weighted as 25%
                            value_count += 0.5
                    else:
                        if k == 0:
                            qa = 'Level Issue'
                        else:
                            qa = 'Flow Issue'                       

                if len(weighting_components) == 0:
                    weighted_color_value = 0
                    weighted_color = ''
                    qa = 'All Data Issue'
                else:
                    sum_color_values = 0
                    for weighting_component in weighting_components:
                        sum_color_values += weighting_component
                    weighted_color_value = sum_color_values / value_count
                    weighted_color_value = round((weighted_color_value+0.001)*2,0)/2 #2.25 should round up to 2.5 as in Excel
                    weighted_color = color_dict[weighted_color_value]
                zone_review.append([event_category,zone,weighted_color,qa])

    result_points = pd.DataFrame(result_points,columns=['Gauge','X','Y','Label','SimType','ResultType','Color'])
    result_points.to_csv(output_folder + '\\ResultPoints.csv',index=False)

    zone_review = pd.DataFrame(zone_review,columns=['Event','Location','Status','QA'])
    for event_category in event_categories:
        event_category_short = event_category[:7]
        event_category_short = event_category_short.replace(' ','_')
        if event_category_short == 'DWF_Cal':
            event_category_short = 'DWF'
        zone_review[zone_review.Event==event_category][['Location','Status','QA']].to_csv(output_folder + '\\Zone_Review_' + event_category_short +'.csv',index=False)




Processing 4HI for confidence maps.


ValueError: index values must be monotonic

In [172]:
flow_df_all.query("(MUID=='Orifice:Highbury_to_4th_Siphon_1' | MUID=='Orifice:Highbury_to_4th_Siphon_2' | MUID=='Weir:ADO to EBO Diversion Weir' | \
    MUID=='Weir:Highbury&EBI overflow to EBO' | MUID=='Weir:Highbury&EBI_Overflow_Weir_2' | MUID=='Weir:SPANISH BANKS PS OVERFLOW' | \
    MUID=='Discovery Outfall') & ResultFile == 'WWF_Calib_Dec_11_2018_4_Days_Base.res1d'").to_excel('Test_Negative.xlsx')